In [1]:
# Forecast and Save to SQLite 

import pandas as pd
import numpy as np
import yfinance as yf
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import keras
import sqlite3
from datetime import datetime

# Define your stock options here
STOCKS = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "AMZN": "Amazon.com, Inc.",
    "GOOGL": "Alphabet Inc. (Class A)",
    "META": "Meta Platforms, Inc.",
    "NVDA": "NVIDIA Corporation",
    "TSLA": "Tesla, Inc.",
    "JPM": "JPMorgan Chase & Co.",
    "BRK-B": "Berkshire Hathaway Inc. (Class B)",
    "V": "Visa Inc.",
    "TCS.NS": "Tata Consultancy Services",
    "INFY.NS": "Infosys Ltd",
    "HDFCBANK.NS": "HDFC Bank Ltd",
    "SBIN.NS": "State Bank of India",
    "RELIANCE.NS": "Reliance Industries Ltd"
}

START_DATE = '2010-01-01'
END_DATE = '2025-06-30'

model = load_model("seq2seq_lstm_stock_forecast.h5", compile=False)
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

# Create SQLite DB
conn = sqlite3.connect('stock_forecasts.db')
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS forecasts (
    stock TEXT,
    company TEXT,
    days_ahead INTEGER,
    forecast_date TEXT,
    predicted_price REAL
)
''')

for ticker, company in STOCKS.items():
    print(f"Processing {ticker} - {company}")
    df = yf.download(ticker, start=START_DATE, end=END_DATE)
    df = df[['Close']].dropna()

    if df.empty or len(df) < 100:
        print(f"Skipping {ticker} - insufficient data")
        continue

    scaler = MinMaxScaler()
    scaled_close = scaler.fit_transform(df[['Close']])
    lookback_days = 100
    forecast_days = 90
    last_seq = scaled_close[-lookback_days:].reshape(1, lookback_days, 1)
    predicted_scaled = model.predict(last_seq)
    predicted_prices = scaler.inverse_transform(predicted_scaled.reshape(-1, 1)).flatten()
    future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=forecast_days, freq='B')

    for horizon in [30, 60, 90]:
        for i in range(horizon):
            cur.execute(
                "INSERT INTO forecasts (stock, company, days_ahead, forecast_date, predicted_price) VALUES (?, ?, ?, ?, ?)",
                (ticker, company, horizon, str(future_dates[i].date()), float(predicted_prices[i]))
            )

conn.commit()
conn.close()
print("Forecasts for all stocks saved in SQLite database 'stock_forecasts.db'.")


Processing AAPL - Apple Inc.


C:\Users\User\AppData\Local\Temp\ipykernel_4388\727345175.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processing MSFT - Microsoft Corporation


C:\Users\User\AppData\Local\Temp\ipykernel_4388\727345175.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Processing SBIN.NS - State Bank of India


C:\Users\User\AppData\Local\Temp\ipykernel_4388\727345175.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Processing RELIANCE.NS - Reliance Industries Ltd


C:\Users\User\AppData\Local\Temp\ipykernel_4388\727345175.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
Forecasts for all stocks saved in SQLite database 'stock_forecasts.db'.


In [5]:
# forecast and save to Excel and CSV
# --------- Imports ---------
import pandas as pd
import numpy as np
import yfinance as yf
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import keras

# --------- 1. Stock List ---------
# Dictionary of stock symbols and their company names
STOCKS = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "AMZN": "Amazon.com, Inc.",
    "GOOGL": "Alphabet Inc. (Class A)",
    "META": "Meta Platforms, Inc.",
    "NVDA": "NVIDIA Corporation",
    "TSLA": "Tesla, Inc.",
    "JPM": "JPMorgan Chase & Co.",
    "BRK-B": "Berkshire Hathaway Inc. (Class B)",
    "V": "Visa Inc.",
    "TCS.NS": "Tata Consultancy Services",
    "INFY.NS": "Infosys Ltd",
    "HDFCBANK.NS": "HDFC Bank Ltd",
    "SBIN.NS": "State Bank of India",
    "RELIANCE.NS": "Reliance Industries Ltd"
}

# --------- 2. Settings ---------
START_DATE = '2010-01-01'
END_DATE = '2025-06-30'
LOOKBACK_DAYS = 100   # Days used for LSTM input
FORECAST_DAYS = 90    # How many days ahead to predict
FORECAST_HORIZONS = [30, 60, 90]  # Which forecast points to extract

# --------- 3. Load the LSTM Model ---------
# Load your pre-trained seq2seq LSTM model
model = load_model("seq2seq_lstm_stock_forecast.h5", compile=False)
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

# --------- 4. Loop Through Stocks and Forecast ---------
results = []  # List to store all results

for ticker, company in STOCKS.items():
    print(f"\nProcessing {ticker} - {company}")

    # 4a. Download historical data
    df = yf.download(ticker, start=START_DATE, end=END_DATE)
    df = df[['Close']].dropna()

    # 4b. Check for enough data
    if df.empty or len(df) < LOOKBACK_DAYS:
        print(f"  Skipping {ticker}: not enough data ({len(df)} rows)")
        continue

    # 4c. Scale closing prices for LSTM
    scaler = MinMaxScaler()
    scaled_close = scaler.fit_transform(df[['Close']])

    # 4d. Prepare input for prediction: last LOOKBACK_DAYS
    last_seq = scaled_close[-LOOKBACK_DAYS:].reshape(1, LOOKBACK_DAYS, 1)

    # 4e. Forecast next FORECAST_DAYS
    pred_scaled = model.predict(last_seq)
    pred_prices = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()

    # 4f. Build list of future business dates
    future_dates = pd.date_range(
        start=df.index[-1] + pd.Timedelta(days=1),
        periods=FORECAST_DAYS,
        freq='B'
    )

    # 4g. Extract only the 30th, 60th, and 90th day forecasts
    for horizon in FORECAST_HORIZONS:
        idx = horizon - 1  # Python is zero-indexed
        results.append({
            "Stock": ticker,
            "Company": company,
            "Horizon": f"{horizon}-Day",
            "Forecast Date": future_dates[idx].date(),
            "Predicted Price": round(pred_prices[idx], 2)
        })

    print(f"  Done. Extracted forecasts for days: {FORECAST_HORIZONS}")

# --------- 5. Save Results to Excel and CSV ---------
df_out = pd.DataFrame(results)
df_out = df_out[["Stock", "Company", "Horizon", "Forecast Date", "Predicted Price"]]  # Order columns

# Save as Excel
df_out.to_excel("forecast_30_60_90.xlsx", index=False)
# Save as CSV
df_out.to_csv("forecast_30_60_90.csv", index=False)

print("\nAll done!")
print("Results saved as forecast_30_60_90.xlsx and forecast_30_60_90.csv")



Processing AAPL - Apple Inc.


C:\Users\User\AppData\Local\Temp\ipykernel_13900\947995425.py:49: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
  Done. Extracted forecasts for days: [30, 60, 90]

Processing MSFT - Microsoft Corporation


C:\Users\User\AppData\Local\Temp\ipykernel_13900\947995425.py:49: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
  Done. Extracted forecasts for days: [30, 60, 90]

Processing AMZN - Amazon.com, Inc.


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
  Done. Extracted forecasts for days: [30, 60, 90]

Processing SBIN.NS - State Bank of India


C:\Users\User\AppData\Local\Temp\ipykernel_13900\947995425.py:49: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
  Done. Extracted forecasts for days: [30, 60, 90]

Processing RELIANCE.NS - Reliance Industries Ltd


C:\Users\User\AppData\Local\Temp\ipykernel_13900\947995425.py:49: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
  Done. Extracted forecasts for days: [30, 60, 90]

All done!
Results saved as forecast_30_60_90.xlsx and forecast_30_60_90.csv


In [6]:
from twilio.rest import Client

account_sid = 'ACbe57d18217f5f3c6aaff8f9b20f14cc6'
auth_token = '764a11b5a66fda908ca2f8a8d7ef5434'
client = Client(account_sid, auth_token)

from_whatsapp_number = 'whatsapp:+14155238886' # Twilio Sandbox WhatsApp number
to_whatsapp_number = 'whatsapp:+447552330220' # Your own WhatsApp number

try:
    message = client.messages.create(
        body="This is test message using Twilio WhatsApp API",
        from_=from_whatsapp_number,
        to=to_whatsapp_number
    )
    print("WhatsApp message sent!")
except Exception as e:
    print("Error sending WhatsApp message:", e)


WhatsApp message sent!


In [3]:
from twilio.rest import Client

account_sid = 'ACbe57d18217f5f3c6aaff8f9b20f14cc6'
auth_token = '764a11b5a66fda908ca2f8a8d7ef5434'
client = Client(account_sid, auth_token)

from_whatsapp_number = 'whatsapp:+14155238886'  # Twilio Sandbox WhatsApp number
to_whatsapp_number = 'whatsapp:+919368173647'   # Your own WhatsApp number

url = 'https://your-server.com/forecast_30_60_90.xlsx'

try:
    message = client.messages.create(
        body=f"Hello Mohit, here is your file: {url}",
        from_=from_whatsapp_number,
        to=to_whatsapp_number
    )
    print("WhatsApp message sent!")
except Exception as e:
    print("Error sending WhatsApp message:", e)

WhatsApp message sent!


In [4]:
from twilio.rest import Client

account_sid = 'ACbe57d18217f5f3c6aaff8f9b20f14cc6'
auth_token = '764a11b5a66fda908ca2f8a8d7ef5434'
client = Client(account_sid, auth_token)

from_whatsapp_number = 'whatsapp:+14155238886'  # Twilio Sandbox WhatsApp number

# List of recipient WhatsApp numbers
to_numbers = [
    'whatsapp:+919368173647',
    'whatsapp:+447552330220',
    # Add more numbers as needed
]

url = 'https://your-server.com/forecast_30_60_90.xlsx'

for to_whatsapp_number in to_numbers:
    try:
        message = client.messages.create(
            body=f"Hello, here is your file: {url}",
            from_=from_whatsapp_number,
            to=to_whatsapp_number
        )
        print(f"WhatsApp message sent to {to_whatsapp_number}!")
    except Exception as e:
        print(f"Error sending WhatsApp message to {to_whatsapp_number}:", e)

WhatsApp message sent to whatsapp:+919368173647!
WhatsApp message sent to whatsapp:+447552330220!
